In this notebook we will be building and training LSTM to predict IBM stock. We will use PyTorch.

## 1. Libraries and settings

In [2]:
import numpy as np
import random
import pandas as pd 
from pylab import mpl, plt
plt.style.use('seaborn-v0_8-darkgrid')
mpl.rcParams['font.family'] = 'serif'
%matplotlib inline

import math, time
import itertools
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
import torch
import torch.nn as nn
from torch.autograd import Variable




## 2. Load data

In [3]:
def stocks_data(symbols, dates):
    df = pd.DataFrame(index=dates)
    for symbol in symbols:
        df_temp = pd.read_csv("../input/Data/Stocks/{}.us.txt".format(symbol), index_col='Date',
                parse_dates=True, usecols=['Date', 'Close'], na_values=['nan'])
        df_temp = df_temp.rename(columns={'Close': symbol})
        df = df.join(df_temp)
    return df

In [4]:
df = pd.read_csv('2ySOLdata1h.csv')
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')
df.set_index('timestamp', inplace=True)
df.drop(columns=df.columns.difference(['Close']), inplace=True)
df.fillna(method='pad')
df

/var/folders/8h/kn141vz15zx75vwjdp__w9fr0000gn/T/ipykernel_13843/2938606158.py:5: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df.fillna(method='pad')


,Close
timestamp,
2022-01-01 04:00:00,172.930
2022-01-01 05:00:00,171.470
2022-01-01 06:00:00,173.220
2022-01-01 07:00:00,172.470
2022-01-01 08:00:00,173.160
...,...
2023-12-31 02:00:00,101.411
2023-12-31 03:00:00,100.738
2023-12-31 04:00:00,100.743


In [5]:
df.head()

,Close
timestamp,
2022-01-01 04:00:00,172.93
2022-01-01 05:00:00,171.47
2022-01-01 06:00:00,173.22
2022-01-01 07:00:00,172.47
2022-01-01 08:00:00,173.16


In [6]:
df_sol=df[['Close']]
# df_sol.values

In [7]:
df_sol=df_sol.fillna(method='ffill')

scaler = MinMaxScaler(feature_range=(-1, 1))
df_sol['Close'] = scaler.fit_transform(df_sol['Close'].values.reshape(-1,1))
#df_ibm
df_sol

/var/folders/8h/kn141vz15zx75vwjdp__w9fr0000gn/T/ipykernel_13843/4009166779.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_sol=df_sol.fillna(method='ffill')


,Close
timestamp,
2022-01-01 04:00:00,0.928339
2022-01-01 05:00:00,0.911244
2022-01-01 06:00:00,0.931735
2022-01-01 07:00:00,0.922953
2022-01-01 08:00:00,0.931032
...,...
2023-12-31 02:00:00,0.090905
2023-12-31 03:00:00,0.083025
2023-12-31 04:00:00,0.083083


In [8]:
# function to create train, test data given stock data and sequence length
def load_data(stock, look_back):
    data_raw = stock.values # convert to numpy array
    data = []
    
    # create all possible sequences of length look_back
    for index in range(len(data_raw) - look_back): 
        data.append(data_raw[index: index + look_back])
    
    data = np.array(data);
    test_set_size = int(np.round(0.2*data.shape[0]));
    train_set_size = data.shape[0] - (test_set_size);
    
    x_train = data[:train_set_size,:-1,:]
    y_train = data[:train_set_size,-1,:]
    
    x_test = data[train_set_size:,:-1,:]
    y_test = data[train_set_size:,-1,:]
    
    return [x_train, y_train, x_test, y_test]

look_back = 60 # choose sequence length
x_train, y_train, x_test, y_test = load_data(df_sol, look_back)
print('x_train.shape = ',x_train.shape)
print('y_train.shape = ',y_train.shape)
print('x_test.shape = ',x_test.shape)
print('y_test.shape = ',y_test.shape)

x_train.shape =  (13951, 59, 1)
y_train.shape =  (13951, 1)
x_test.shape =  (3488, 59, 1)
y_test.shape =  (3488, 1)


In [9]:
y_train

array([[ 0.93981441],
       [ 0.90597465],
       [ 0.89707561],
       ...,
       [-0.82899798],
       [-0.82770996],
       [-0.82713621]])

In [10]:
print('x_train.shape = ',x_train.dtype)
print('y_train.shape = ',y_train.dtype)
print('x_test.shape = ',x_test.dtype)
print('y_test.shape = ',y_test.dtype)

x_train.shape =  float64
y_train.shape =  float64
x_test.shape =  float64
y_test.shape =  float64


In [11]:
# make training and test sets in torch
x_train = torch.from_numpy(x_train).type(torch.Tensor)
x_test = torch.from_numpy(x_test).type(torch.Tensor)
y_train = torch.from_numpy(y_train).type(torch.Tensor)
y_test = torch.from_numpy(y_test).type(torch.Tensor)

In [12]:
y_test

tensor([[-0.8270],
        [-0.8270],
        [-0.8260],
        ...,
        [ 0.0830],
        [ 0.0831],
        [ 0.0975]])

In [13]:
y_train.size(),x_train.size()

(torch.Size([13951, 1]), torch.Size([13951, 59, 1]))

## 3. Build the structure of model

In [14]:
# Build model
#####################
input_dim = 1
hidden_dim = 32
num_layers = 2 
output_dim = 1


# Here we define our model as a class
class LSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(LSTM, self).__init__()
        # Hidden dimensions
        self.hidden_dim = hidden_dim

        # Number of hidden layers
        self.num_layers = num_layers

        # batch_first=True causes input/output tensors to be of shape
        # (batch_dim, seq_dim, feature_dim)
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)

        # Readout layer
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # Initialize cell state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_dim).requires_grad_()

        # We need to detach as we are doing truncated backpropagation through time (BPTT)
        # If we don't, we'll backprop all the way to the start even after going through another batch
        out, (hn, cn) = self.lstm(x, (h0.detach(), c0.detach()))

        # Index hidden state of last time step
        # out.size() --> 100, 32, 100
        # out[:, -1, :] --> 100, 100 --> just want last time step hidden states! 
        out = self.fc(out[:, -1, :]) 
        # out.size() --> 100, 10
        return out
    
model = LSTM(input_dim=input_dim, hidden_dim=hidden_dim, output_dim=output_dim, num_layers=num_layers)

loss_fn = torch.nn.MSELoss()

optimiser = torch.optim.Adam(model.parameters(), lr=0.01)
print(model)
print(len(list(model.parameters())))
for i in range(len(list(model.parameters()))):
    print(list(model.parameters())[i].size())

LSTM(
  (lstm): LSTM(1, 32, num_layers=2, batch_first=True)
  (fc): Linear(in_features=32, out_features=1, bias=True)
)
10
torch.Size([128, 1])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])
torch.Size([128, 32])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])
torch.Size([1, 32])
torch.Size([1])


In [15]:
look_back-1

59

In [16]:
x_train.shape

torch.Size([13951, 59, 1])

In [17]:
# Train model
#####################
num_epochs = 100
hist = np.zeros(num_epochs)

# Number of steps to unroll
seq_dim =look_back-1  

for t in range(num_epochs):
    # Initialise hidden state
    # Don't do this if you want your LSTM to be stateful
    #model.hidden = model.init_hidden()
    
    # Forward pass
    y_train_pred = model(x_train)

    loss = loss_fn(y_train_pred, y_train)
    # if t % 10 == 0 and t !=0:
    print("Epoch ", t, "MSE: ", loss.item())
    hist[t] = loss.item()

    # Zero out gradient, else they will accumulate between epochs
    optimiser.zero_grad()

    # Backward pass
    loss.backward()

    # Update parameters
    optimiser.step()

Epoch  0 MSE:  0.6694104671478271
Epoch  1 MSE:  0.4961107671260834
Epoch  2 MSE:  0.3147190511226654
Epoch  3 MSE:  0.1414676159620285
Epoch  4 MSE:  0.32508939504623413
Epoch  5 MSE:  0.2028154581785202
Epoch  6 MSE:  0.12628689408302307
Epoch  7 MSE:  0.12543046474456787
Epoch  8 MSE:  0.1437053382396698
Epoch  9 MSE:  0.15258166193962097
Epoch  10 MSE:  0.1465945541858673
Epoch  11 MSE:  0.1291397213935852
Epoch  12 MSE:  0.10646167397499084
Epoch  13 MSE:  0.08536248654127121
Epoch  14 MSE:  0.07076741009950638
Epoch  15 MSE:  0.06245074048638344
Epoch  16 MSE:  0.05407920107245445
Epoch  17 MSE:  0.038306526839733124
Epoch  18 MSE:  0.018467677757143974
Epoch  19 MSE:  0.008549815975129604
Epoch  20 MSE:  0.012376506812870502
Epoch  21 MSE:  0.022639276459813118
Epoch  22 MSE:  0.02753991074860096
Epoch  23 MSE:  0.020070461556315422
Epoch  24 MSE:  0.013301336206495762
Epoch  25 MSE:  0.012487824074923992
Epoch  26 MSE:  0.01246095634996891
Epoch  27 MSE:  0.009768710471689701
E

In [33]:
# plt.plot(hist, label="Training loss")
# plt.legend()
# plt.show()

In [20]:
# make predictions
y_test_pred = model(x_test)

# invert predictions
y_train_pred = scaler.inverse_transform(y_train_pred.detach().numpy())
y_train = scaler.inverse_transform(y_train.detach().numpy())
y_test_pred = scaler.inverse_transform(y_test_pred.detach().numpy())
y_test = scaler.inverse_transform(y_test.detach().numpy())

# calculate root mean squared error
trainScore = math.sqrt(mean_squared_error(y_train[:,0], y_train_pred[:,0]))
print('Train Score: %.2f RMSE' % (trainScore))
testScore = math.sqrt(mean_squared_error(y_test[:,0], y_test_pred[:,0]))
print('Test Score: %.2f RMSE' % (testScore))

Train Score: 1.23 RMSE
Test Score: 1.21 RMSE


In [21]:
import vectorbtpro as vbt
from plotly.subplots import make_subplots
import plotly.graph_objects as go
vbt.settings.set_theme('dark')
vbt.settings['plotting']['layout']['width'] = 800
vbt.settings['plotting']['layout']['height'] = 400

In [31]:
# Assuming df_sol is your original DataFrame and y_test, y_test_pred are your numpy arrays
# First, create the index you want to use for the x-axis
x_axis_index = df[len(df)-len(y_test):].index

# Create Series with the custom index
y_test_series = pd.Series(y_test.flatten(), index=x_axis_index, name="Actual")
y_test_pred_series = pd.Series(y_test_pred.flatten(), index=x_axis_index, name="Predicted")

# Create a DataFrame from your series
combined_df = pd.DataFrame({
    "Actual": y_test_series,
    "Predicted": y_test_pred_series
})
entries = 2
exits = 0

combined_df['Actual_Diff'] = combined_df['Actual'].diff()
combined_df['Predicted_Diff'] = combined_df['Predicted'].diff()


cross_over = (combined_df['Actual_Diff'] > 0) & (combined_df['Actual'] > combined_df['Predicted']) & (combined_df['Actual'].shift(1) <= combined_df['Predicted'].shift(1))
cross_under = (combined_df['Actual_Diff'] < 0) & (combined_df['Actual'] < combined_df['Predicted']) & (combined_df['Actual'].shift(1) >= combined_df['Predicted'].shift(1))

combined_df['Signal'] = 1  # Default to '1' for hold/no action
combined_df.loc[cross_over, 'Signal'] = 2  # '2' for cross over
combined_df.loc[cross_under, 'Signal'] = 0  # '0' for cross under


# Plot using vectorbt
combined_df_vbt = vbt.Data.from_data(combined_df)
fig = combined_df_vbt.plot(trace_kwargs=dict(mode='lines'))
fig.show()

In [32]:
signal = combined_df['Signal']
entries = signal == 2
exits = signal == 0

pf = vbt.Portfolio.from_signals(
    close=combined_df.Actual, 
    long_entries=entries, 
    long_exits=exits,
    size=100,
    size_type='value',
    init_cash='auto'
)

pf.plot({"orders", "cum_returns", }, settings=dict(bm_returns=False)).show()